In [1]:
from word2vec import *

In [2]:
words = ['the', 'quick', 'brown', 'fox', 'jumps', 'over', 'the', 'lazy', 'dog']
vocab = set(words)
embedding_size = 10

word2vec = Word2Vec(vocab, embedding_size)

## Backward
This is the forward graph
```
embedding --(@ linear)--> x --(softmax)--> o --(criterion)--> loss
```

Gradient descent step

In [3]:
input_index, output_index = 4, 1

for _ in range(20):
    word2vec.step(input_index, output_index)
_, output = word2vec.forward(input_index)
output

Loss - 1.206
Loss - 0.788
Loss - 0.472
Loss - 0.225
Loss - 0.080
Loss - 0.036
Loss - 0.023
Loss - 0.017
Loss - 0.013
Loss - 0.011
Loss - 0.009
Loss - 0.008
Loss - 0.007
Loss - 0.006
Loss - 0.006
Loss - 0.005
Loss - 0.005
Loss - 0.004
Loss - 0.004
Loss - 0.004


array([3.44650667e-03, 9.72332704e-01, 5.55491309e-03, 4.73745916e-03,
       7.21280904e-03, 2.88353822e-03, 3.35513047e-04, 3.49655686e-03])

It learns very well!